In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('card_transaction.csv')

df_part1, df_part2 = np.array_split(df, 2)

df_part1.to_csv('card_transaction_part1.csv', index=False)
df_part2.to_csv('card_transaction_part2.csv', index=False)


In [1]:
import sys
!{sys.executable} -m pip install delta-spark==3.2.0

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable

spark = SparkSession.builder \
    .appName("DeltaDynamicSchemaTask") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "matrix") \
    .config("spark.hadoop.fs.s3a.secret.key", "matrix123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print("Spark Sessiyası Yaradıldı")

Spark Sessiyası Yaradıldı


In [ ]:
delta_path = "s3a://silver/card_transactions_delta"

df1 = spark.read.option("header", "true").option("inferSchema", "true").csv("card_transaction_part1.csv")
df1.write.format("delta").mode("overwrite").partitionBy("transaction_date").save(delta_path)

df2_raw = spark.read.option("header", "true").csv("card_transaction_part2.csv")

dt = DeltaTable.forPath(spark, delta_path)
target_schema = dt.toDF().schema
partition_cols = dt.detail().select("partitionColumns").collect()[0][0]

for field in target_schema:
    df2_raw = df2_raw.withColumn(field.name, col(field.name).cast(field.dataType))

df2_raw.write.format("delta").mode("append").partitionBy(*partition_cols).save(delta_path)

spark.read.format("delta").load(delta_path).show(5)